In [1]:
import pandas as pd

In [ ]:
# import os
# import time
# import pandas as pd
# from datetime import datetime, timedelta
# from binance.client import Client
# from binance.exceptions import BinanceAPIException
# from requests.exceptions import ConnectionError

# # ── 1. CONFIGURATION ─────────────────────────────────────────────────────────
# API_KEY    = "YOUR_BINANCE_API_KEY"       # <<<–– letakkan API key Anda
# API_SECRET = "YOUR_BINANCE_API_SECRET"    # <<<–– letakkan API secret Anda
# SYMBOL     = "BTCUSDT"
# INTERVAL   = Client.KLINE_INTERVAL_1HOUR  # hourly
# START      = "1 Jan, 2017"                # tarik data sejak listing

# # Delay antar panggilan (detik)
# DELAY = 5
# # Jumlah retry maksimum
# MAX_RETRIES = 5

# client = Client(API_KEY, API_SECRET)

# # ── 2. SAFE WRAPPERS ──────────────────────────────────────────────────────────
# def retry_api(func, *args, **kwargs):
#     backoff = 1
#     for i in range(MAX_RETRIES):
#         try:
#             result = func(*args, **kwargs)
#             time.sleep(DELAY)
#             return result
#         except (ConnectionError, BinanceAPIException) as e:
#             print(f"⚠️ API error ({e}), retry {i+1}/{MAX_RETRIES} in {backoff}s…")
#             time.sleep(backoff)
#             backoff *= 2
#     raise RuntimeError(f"Gagal setelah {MAX_RETRIES} retry pada {func.__name__}")

# def safe_get_klines(**kwargs):
#     return retry_api(client.get_klines, **kwargs)

# def safe_futures_funding_rate(**kwargs):
#     return retry_api(client.futures_funding_rate, **kwargs)

# def safe_aggregate_trades(**kwargs):
#     return retry_api(client.aggregate_trades, **kwargs)

# def safe_open_interest_history(**kwargs):
#     return retry_api(client.futures_open_interest_history, **kwargs)

# def safe_get_order_book(**kwargs):
#     return retry_api(client.get_order_book, **kwargs)

# # ── 3. FETCH FUNCTIONS ────────────────────────────────────────────────────────
# def fetch_full_ohlcv(symbol, interval, start_str):
#     all_df = []
#     last_time = pd.to_datetime(start_str)
#     while True:
#         klines = safe_get_klines(
#             symbol=symbol,
#             interval=interval,
#             startTime=int(last_time.timestamp()*1000),
#             limit=1000
#         )
#         if not klines:
#             break
#         df = pd.DataFrame(klines, columns=[
#             "open_time","open","high","low","close","volume",
#             "close_time","quote_asset_volume","num_trades",
#             "taker_buy_base_vol","taker_buy_quote_vol","ignore"
#         ])
#         df["open_time"]  = pd.to_datetime(df["open_time"], unit="ms")
#         df["close_time"] = pd.to_datetime(df["close_time"], unit="ms")
#         df = df.rename(columns={"open_time":"timestamp"})
#         all_df.append(df)
#         last_time = df["timestamp"].iloc[-1] + timedelta(milliseconds=1)
#         if last_time >= datetime.utcnow():
#             break
#     return pd.concat(all_df, ignore_index=True)

# def fetch_full_funding(symbol, start_str):
#     all_df = []
#     last_time = pd.to_datetime(start_str)
#     while True:
#         fr = safe_futures_funding_rate(
#             symbol=symbol,
#             startTime=int(last_time.timestamp()*1000),
#             limit=1000
#         )
#         if not fr:
#             break
#         df = pd.DataFrame(fr)
#         df["fundingTime"] = pd.to_datetime(df["fundingTime"], unit="ms")
#         df["fundingRate"] = df["fundingRate"].astype(float)
#         all_df.append(df[["fundingTime","fundingRate"]])
#         last_time = df["fundingTime"].iloc[-1] + timedelta(milliseconds=1)
#         if last_time >= datetime.utcnow():
#             break
#     df_full = pd.concat(all_df, ignore_index=True)
#     # resample per jam: ambil last fundingRate setiap jam
#     df_full = (
#         df_full
#         .set_index("fundingTime")
#         .resample("1H").last()
#         .dropna()
#         .reset_index()
#         .rename(columns={"fundingTime":"timestamp"})
#     )
#     return df_full

# def fetch_full_agg_trades(symbol, start_str):
#     all_trades = []
#     last_id = None
#     start_ms = int(pd.to_datetime(start_str).timestamp()*1000)
#     while True:
#         params = {"symbol":symbol, "limit":1000, "startTime":start_ms}
#         if last_id:
#             params["fromId"] = last_id + 1
#         trades = safe_aggregate_trades(**params)
#         if not trades:
#             break
#         df = pd.DataFrame(trades).rename(
#             columns={"a":"aggTradeId","T":"tradeTime","q":"qty"}
#         )
#         df["tradeTime"] = pd.to_datetime(df["tradeTime"], unit="ms")
#         all_trades.append(df)
#         last_id = df["aggTradeId"].iloc[-1]
#     df_full = pd.concat(all_trades, ignore_index=True)
#     df_full["hour"] = df_full["tradeTime"].dt.floor("H")
#     agg = (
#         df_full.groupby("hour")
#         .apply(lambda d: pd.Series({
#             "buy_vol":  d.loc[~d["isBuyerMaker"], "qty"].sum(),
#             "sell_vol": d.loc[ d["isBuyerMaker"], "qty"].sum()
#         }))
#         .reset_index()
#         .rename(columns={"hour":"timestamp"})
#     )
#     return agg

# def fetch_full_open_interest(symbol, start_str):
#     all_df = []
#     last_time = pd.to_datetime(start_str)
#     while True:
#         hist = safe_open_interest_history(
#             symbol=symbol,
#             period="1h",
#             startTime=int(last_time.timestamp()*1000),
#             limit=1000
#         )
#         if not hist:
#             break
#         df = pd.DataFrame(hist)
#         df["timestamp"]     = pd.to_datetime(df["timestamp"], unit="ms")
#         df["openInterest"] = df["sumOpenInterest"].astype(float)
#         all_df.append(df[["timestamp","openInterest"]])
#         last_time = df["timestamp"].iloc[-1] + timedelta(milliseconds=1)
#         if last_time >= datetime.utcnow():
#             break
#     return pd.concat(all_df, ignore_index=True)

# def fetch_order_book_snapshot(symbol, limit=100):
#     ob = safe_get_order_book(symbol=symbol, limit=limit)
#     bids = pd.DataFrame(ob["bids"], columns=["price","qty"]).astype(float)
#     asks = pd.DataFrame(ob["asks"], columns=["price","qty"]).astype(float)
#     now = datetime.utcnow().replace(minute=0,second=0,microsecond=0)
#     return pd.DataFrame([{
#         "timestamp":   now,
#         "bid_depth":   bids["qty"].sum(),
#         "ask_depth":   asks["qty"].sum(),
#         "top_bid":     bids["price"].iloc[0],
#         "top_ask":     asks["price"].iloc[0]
#     }])

# # ── 4. MAIN: EKSEKUSI & SIMPAN CSV ────────────────────────────────────────────
# if __name__ == "__main__":
#     os.makedirs("data", exist_ok=True)

#     # 4.1 OHLCV
#     df_ohlcv = fetch_full_ohlcv(SYMBOL, INTERVAL, START)
#     df_ohlcv.to_csv("data/ohlcv.csv", index=False)

#     # 4.2 Funding Rate
#     df_fr = fetch_full_funding(SYMBOL, START)
#     df_fr.to_csv("data/funding_rate.csv", index=False)

#     # 4.3 Aggregate Trades
#     df_trades = fetch_full_agg_trades(SYMBOL, START)
#     df_trades.to_csv("data/agg_trades.csv", index=False)

#     # 4.4 Open Interest
#     df_oi = fetch_full_open_interest(SYMBOL, START)
#     df_oi.to_csv("data/open_interest.csv", index=False)

#     # 4.5 Order Book snapshot
#     df_ob = fetch_order_book_snapshot(SYMBOL, limit=100)
#     df_ob.to_csv("data/order_book.csv", index=False)

#     # 4.6 Merge semua berdasarkan timestamp (hourly)
#     dfs = [
#         df_ohlcv.set_index("timestamp"),
#         df_fr.set_index("timestamp"),
#         df_trades.set_index("timestamp"),
#         df_oi.set_index("timestamp"),
#         df_ob.set_index("timestamp")
#     ]
#     df_combined = pd.concat(dfs, axis=1).reset_index()
#     df_combined.to_csv("data/combined.csv", index=False)

#     print("✅ Semua data off-chain per jam berhasil diambil, di-retry jika error, dan disimpan di ./data/") 


Block 1 – fetch_ohlcv.py

In [7]:
import os
import time
import pandas as pd
from datetime import datetime, timedelta
from requests.exceptions import ConnectionError
from binance.client import Client
from binance.exceptions import BinanceAPIException

# ── CONFIG ───────────────────────────────────────────────────────────────────
API_KEY    = "H2u6FGfNoMQTzzjjNtnRhMHJ5gQ2oPdWGV4RYNwGmyz2854hCS9pssxYm0nxEefn"
API_SECRET = "u8ByWKpZX1sgTNVGZMzCwoHWkiQlRKIKMLnkNIBVtORNRx1vWlZNRsKvigHa2ylr"
SYMBOL     = "BTCUSDT"
INTERVAL   = Client.KLINE_INTERVAL_1HOUR  # hourly
START      = "1 Jan, 2017"
DELAY      = 0.2
MAX_RETRIES= 5

client = Client(API_KEY, API_SECRET)

# ── SAFE WRAPPER ─────────────────────────────────────────────────────────────
def retry_api(func, *args, **kwargs):
    backoff = 1
    for i in range(MAX_RETRIES):
        try:
            result = func(*args, **kwargs)
            time.sleep(DELAY)
            return result
        except (ConnectionError, BinanceAPIException) as e:
            print(f"⚠️ {func.__name__} error ({e}), retry {i+1}/{MAX_RETRIES} in {backoff}s")
            time.sleep(backoff)
            backoff *= 2
    raise RuntimeError(f"Failed after {MAX_RETRIES} retries in {func.__name__}")

def safe_get_klines(**kwargs):
    return retry_api(client.get_klines, **kwargs)

# ── FETCH FUNCTION ───────────────────────────────────────────────────────────
def fetch_full_ohlcv(symbol, interval, start_str):
    all_df   = []
    last_ts  = pd.to_datetime(start_str)
    while True:
        klines = safe_get_klines(
            symbol=symbol,
            interval=interval,
            startTime=int(last_ts.timestamp()*1000),
            limit=1000
        )
        if not klines:
            break
        df = pd.DataFrame(klines, columns=[
            "open_time","open","high","low","close","volume",
            "close_time","quote_asset_volume","num_trades",
            "taker_buy_base_vol","taker_buy_quote_vol","ignore"
        ])
        df["open_time"]  = pd.to_datetime(df["open_time"], unit="ms")
        df = df.rename(columns={"open_time":"timestamp"})
        all_df.append(df)
        last_ts = df["timestamp"].iloc[-1] + timedelta(milliseconds=1)
        if last_ts >= datetime.utcnow():
            break
    return pd.concat(all_df, ignore_index=True)

# ── MAIN ─────────────────────────────────────────────────────────────────────
if __name__ == "__main__":
    os.makedirs("data", exist_ok=True)
    df = fetch_full_ohlcv(SYMBOL, INTERVAL, START)
    df.to_csv("data2/ohlcv.csv", index=False)
    print(f"✅ OHLCV saved: {len(df)} bars")


✅ OHLCV saved: 68324 bars


Block 2 – fetch_funding_rate.py

In [ ]:
# import os
# import time
# import pandas as pd
# from datetime import datetime, timedelta
# from requests.exceptions import ConnectionError
# from binance.client import Client
# from binance.exceptions import BinanceAPIException

# # ── CONFIG ───────────────────────────────────────────────────────────────────
# API_KEY    = "H2u6FGfNoMQTzzjjNtnRhMHJ5gQ2oPdWGV4RYNwGmyz2854hCS9pssxYm0nxEefn"
# API_SECRET = "u8ByWKpZX1sgTNVGZMzCwoHWkiQlRKIKMLnkNIBVtORNRx1vWlZNRsKvigHa2ylr"
# SYMBOL     = "BTCUSDT"
# START      = "1 Jan, 2017"
# DELAY      = 0.2
# MAX_RETRIES= 5

# client = Client(API_KEY, API_SECRET)

# # ── SAFE WRAPPER ─────────────────────────────────────────────────────────────
# def retry_api(func, *args, **kwargs):
#     backoff = 1
#     for i in range(MAX_RETRIES):
#         try:
#             res = func(**kwargs) if func.__name__!="client" else func(*args, **kwargs)
#             time.sleep(DELAY)
#             return res
#         except (ConnectionError, BinanceAPIException) as e:
#             print(f"⚠️ {func.__name__} error ({e}), retry {i+1}/{MAX_RETRIES} in {backoff}s")
#             time.sleep(backoff)
#             backoff *= 2
#     raise RuntimeError(f"Failed after {MAX_RETRIES} retries in {func.__name__}")

# def safe_futures_funding_rate(**kwargs):
#     return retry_api(client.futures_funding_rate, **kwargs)

# # ── FETCH FUNCTION ───────────────────────────────────────────────────────────
# def fetch_full_funding(symbol, start_str):
#     all_df  = []
#     last_ts = pd.to_datetime(start_str)
#     while True:
#         fr = safe_futures_funding_rate(
#             symbol=symbol,
#             startTime=int(last_ts.timestamp()*1000),
#             limit=500
#         )
#         if not fr:
#             break
#         df = pd.DataFrame(fr)
#         df["fundingTime"] = pd.to_datetime(df["fundingTime"], unit="ms")
#         df["fundingRate"] = df["fundingRate"].astype(float)
#         all_df.append(df[["fundingTime","fundingRate"]])
#         last_ts = df["fundingTime"].iloc[-1] + timedelta(milliseconds=1)
#         if last_ts >= datetime.utcnow():
#             break

#     df_full = pd.concat(all_df, ignore_index=True)
#     df_hour = (
#         df_full
#         .set_index("fundingTime")
#         .resample("1H").last()
#         .dropna()
#         .reset_index()
#         .rename(columns={"fundingTime":"timestamp"})
#     )
#     return df_hour

# # ── MAIN ─────────────────────────────────────────────────────────────────────
# if __name__ == "__main__":
#     os.makedirs("data", exist_ok=True)
#     df = fetch_full_funding(SYMBOL, START)
#     df.to_csv("data2/funding_rate.csv", index=False)
#     print(f"✅ Funding Rate saved: {len(df)} records")


✅ Funding Rate saved: 6294 records


C:\Users\Elzandi Irfan Zikra\AppData\Local\Temp\ipykernel_12944\4053613433.py:60: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  .resample("1H").last()


Block 3 – fetch_agg_trades.py

In [ ]:
# # fetch_agg_trades.py

# import os
# import time
# import pandas as pd
# from datetime import datetime
# from requests.exceptions import ConnectionError
# from binance.client import Client
# from binance.exceptions import BinanceAPIException

# # ── CONFIG ───────────────────────────────────────────────────────────────────
# API_KEY    = "H2u6FGfNoMQTzzjjNtnRhMHJ5gQ2oPdWGV4RYNwGmyz2854hCS9pssxYm0nxEefn"
# API_SECRET = "u8ByWKpZX1sgTNVGZMzCwoHWkiQlRKIKMLnkNIBVtORNRx1vWlZNRsKvigHa2ylr"
# SYMBOL      = "BTCUSDT"
# START       = "1 Jan, 2017"
# DELAY       = 0.2
# MAX_RETRIES = 5

# client = Client(API_KEY, API_SECRET)

# # ── SAFE WRAPPER ─────────────────────────────────────────────────────────────
# def retry_api(func, *args, **kwargs):
#     backoff = 1
#     for i in range(MAX_RETRIES):
#         try:
#             res = func(*args, **kwargs)
#             time.sleep(DELAY)
#             return res
#         except (ConnectionError, BinanceAPIException) as e:
#             print(f"⚠️ {func.__name__} error ({e}), retry {i+1}/{MAX_RETRIES} in {backoff}s")
#             time.sleep(backoff)
#             backoff *= 2
#     raise RuntimeError(f"Failed after {MAX_RETRIES} retries in {func.__name__}")

# def safe_get_aggregate_trades(**kwargs):
#     return retry_api(client.get_aggregate_trades, **kwargs)

# # ── FETCH FUNCTION ───────────────────────────────────────────────────────────
# def fetch_full_agg_trades(symbol, start_str):
#     all_trades = []
#     last_id    = None
#     start_ms   = int(pd.to_datetime(start_str).timestamp() * 1000)

#     while True:
#         if last_id is None:
#             params = {
#                 "symbol":    symbol,
#                 "startTime": start_ms,
#                 "limit":     500
#             }
#         else:
#             params = {
#                 "symbol": symbol,
#                 "fromId":  last_id + 1,
#                 "limit":   500
#             }

#         trades = safe_get_aggregate_trades(**params)
#         if not trades:
#             break

#         df = pd.DataFrame(trades).rename(
#             columns={"a": "aggTradeId", "T": "tradeTime", "q": "qty"}
#         )
#         df["tradeTime"] = pd.to_datetime(df["tradeTime"], unit="ms")
#         all_trades.append(df)
#         last_id = df["aggTradeId"].iloc[-1]

#     df_full = pd.concat(all_trades, ignore_index=True)
#     df_full["hour"] = df_full["tradeTime"].dt.floor("H")
#     df_hour = (
#         df_full
#         .groupby("hour")
#         .apply(lambda d: pd.Series({
#             "buy_vol":  d.loc[~d["isBuyerMaker"], "qty"].sum(),
#             "sell_vol": d.loc[ d["isBuyerMaker"], "qty"].sum()
#         }))
#         .reset_index()
#         .rename(columns={"hour": "timestamp"})
#     )
#     return df_hour

# # ── MAIN ─────────────────────────────────────────────────────────────────────
# if __name__ == "__main__":
#     os.makedirs("data", exist_ok=True)
#     df = fetch_full_agg_trades(SYMBOL, START)
#     df.to_csv("data/agg_trades.csv", index=False)
#     print(f"✅ Agg Trades saved: {len(df)} hourly records")


Block 4 – fetch_interest_and_orderbook.py

In [ ]:
# import os
# import time
# import pandas as pd
# from datetime import datetime
# from binance.client import Client
# from binance.exceptions import BinanceAPIException
# from requests.exceptions import ConnectionError

# # ── CONFIG ───────────────────────────────────────────────────────────────────
# API_KEY    = "H2u6FGfNoMQTzzjjNtnRhMHJ5gQ2oPdWGV4RYNwGmyz2854hCS9pssxYm0nxEefn"
# API_SECRET = "u8ByWKpZX1sgTNVGZMzCwoHWkiQlRKIKMLnkNIBVtORNRx1vWlZNRsKvigHa2ylr"
# SYMBOL      = "BTCUSDT"
# DELAY       = 0.2
# MAX_RETRIES = 5

# client = Client(API_KEY, API_SECRET)

# def retry_api(func, **kwargs):
#     backoff = 1
#     for i in range(MAX_RETRIES):
#         try:
#             result = func(**kwargs)
#             time.sleep(DELAY)
#             return result
#         except (ConnectionError, BinanceAPIException) as e:
#             print(f"⚠️ {func.__name__} error ({e}), retry {i+1}/{MAX_RETRIES} in {backoff}s")
#             time.sleep(backoff)
#             backoff *= 2
#     raise RuntimeError(f"Failed after {MAX_RETRIES} retries in {func.__name__}")

# # ── 1. ambil snapshot Open Interest ──────────────────────────────────────────
# def get_open_interest(symbol):
#     oi = retry_api(client.futures_open_interest, symbol=symbol)
#     return float(oi["openInterest"])

# # ── 2. ambil snapshot Order Book ─────────────────────────────────────────────
# def get_order_book(symbol, limit=100):
#     ob = retry_api(client.get_order_book, symbol=symbol, limit=limit)
#     bids = pd.DataFrame(ob["bids"], columns=["price","qty"]).astype(float)
#     asks = pd.DataFrame(ob["asks"], columns=["price","qty"]).astype(float)
#     return bids, asks

# # ── MAIN ─────────────────────────────────────────────────────────────────────
# if __name__ == "__main__":
#     os.makedirs("data", exist_ok=True)

#     now = datetime.utcnow().replace(minute=0, second=0, microsecond=0)

#     # 1) Open Interest
#     oi_val = get_open_interest(SYMBOL)
#     df_oi  = pd.DataFrame([{"timestamp": now, "openInterest": oi_val}])
#     # append (atau buat baru)
#     oi_path = "data/open_interest.csv"
#     if os.path.exists(oi_path):
#         df_oi.to_csv(oi_path, mode="a", header=False, index=False)
#     else:
#         df_oi.to_csv(oi_path, index=False)
#     print(f"✅ Open Interest snapshot saved for {now}")

#     # 2) Order Book
#     bids, asks = get_order_book(SYMBOL, limit=100)
#     df_ob = pd.DataFrame([{
#         "timestamp": now,
#         "bid_depth": bids["qty"].sum(),
#         "ask_depth": asks["qty"].sum(),
#         "top_bid":   bids["price"].iloc[0],
#         "top_ask":   asks["price"].iloc[0]
#     }])
#     ob_path = "data/order_book.csv"
#     if os.path.exists(ob_path):
#         df_ob.to_csv(ob_path, mode="a", header=False, index=False)
#     else:
#         df_ob.to_csv(ob_path, index=False)
#     print(f"✅ Order Book snapshot saved for {now}")


✅ Open Interest snapshot saved for 2025-06-08 08:00:00
✅ Order Book snapshot saved for 2025-06-08 08:00:00


In [ ]:
# # fetch_open_interest_klines.py

# import os
# import time
# import random
# import requests
# import pandas as pd
# from datetime import datetime, timedelta

# # ── CONFIG ───────────────────────────────────────────────────────────────────
# SYMBOL      = "BTCUSDT"
# INTERVAL    = "1h"             # 1h, 4h, 1d, dll.
# START       = "2017-01-01"     # tarik data sejak 1 Jan 2017
# LIMIT       = 1500             # max record per request
# DATA_DIR    = "data"
# MAX_RETRIES = 5
# DELAY       = 0.2              # detik antar request

# os.makedirs(DATA_DIR, exist_ok=True)

# # ── SAFE REQUEST ──────────────────────────────────────────────────────────────
# def safe_request(url, params):
#     backoff = 1
#     for attempt in range(1, MAX_RETRIES+1):
#         try:
#             r = requests.get(url, params=params, timeout=10)
#             r.raise_for_status()
#             # jitter agar pola tidak seragam
#             time.sleep(DELAY + random.random()*0.1)
#             return r.json()
#         except Exception as e:
#             print(f"⚠️ Attempt {attempt}/{MAX_RETRIES} failed: {e}. Retrying in {backoff}s…")
#             time.sleep(backoff)
#             backoff *= 2
#     raise RuntimeError(f"Failed to fetch after {MAX_RETRIES} retries")

# # ── FETCH OPEN INTEREST KLINES ────────────────────────────────────────────────
# def fetch_open_interest_klines(symbol, interval, start_str):
#     url = "https://fapi.binance.com/futures/data/openInterestKlines"
#     start_ts = pd.to_datetime(start_str)
#     all_df   = []

#     while True:
#         params = {
#             "symbol":    symbol,
#             "interval":  interval,
#             "limit":     LIMIT,
#             "startTime": int(start_ts.timestamp() * 1000)
#         }
#         data = safe_request(url, params)
#         if not data:
#             break

#         df = pd.DataFrame(data, columns=[
#             "open_time", "open_interest", "open", "high", "low",
#             "close", "volume", "close_time", "quote_volume", "count"
#         ])
#         df["open_time"]     = pd.to_datetime(df["open_time"], unit="ms")
#         df["open_interest"] = df["open_interest"].astype(float)
#         df = df[["open_time", "open_interest"]]
#         all_df.append(df)

#         last_ts = df["open_time"].iloc[-1]
#         start_ts = last_ts + timedelta(milliseconds=1)
#         if start_ts >= datetime.utcnow():
#             break

#     return pd.concat(all_df, ignore_index=True)

# # ── MAIN ─────────────────────────────────────────────────────────────────────
# if __name__ == "__main__":
#     print("▶️ Fetching Open Interest historis via openInterestKlines…")
#     df_oi = fetch_open_interest_klines(SYMBOL, INTERVAL, START)
#     path  = os.path.join(DATA_DIR, "open_interest_hist.csv")
#     df_oi.to_csv(path, index=False)
#     print(f"✅ Saved {len(df_oi)} records to {path}")
#     print(f"   Range: {df_oi['open_time'].min()} → {df_oi['open_time'].max()}")


On Chain

In [ ]:
# import os
# import time
# import requests
# import pandas as pd
# from datetime import datetime

# # ── 1. LETAKKAN API KEY DI SINI ────────────────────────────────────────────────
# API_KEY = "YOUR_GLASSNODE_API_KEY"      # <<<–– letakkan API key Anda
# ASSET   = "BTC"

# # ── 2. PARAMETER UMUM ─────────────────────────────────────────────────────────
# BASE_URL = "https://api.glassnode.com/v1/metrics"
# START    = "2017-01-01"                 # tarik data sejak 1 Jan 2017
# END      = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")
# RES      = "1h"                         # interval 1 jam

# # konversi ke UNIX timestamp (detik)
# since = int(pd.to_datetime(START).timestamp())
# until = int(pd.to_datetime(END).timestamp())

# # definisi metrics yang ingin di-fetch
# metrics = {
#     "active_addresses":    "addresses/active_count",
#     "exchange_inflow":     "exchanges/inflow",
#     "exchange_outflow":    "exchanges/outflow",
#     "tx_count":            "transactions/count",
#     "miner_exch_vol":      "transactions/transfers_volume_miners_to_exchanges",
#     "whale_tx_count":      "transactions/transfers_exchanges_to_whales_count"
# }

# os.makedirs("data", exist_ok=True)

# for name, path in metrics.items():
#     url = f"{BASE_URL}/{path}"
#     params = {
#         "a":        ASSET,
#         "s":        since,
#         "u":        until,
#         "i":        RES,
#         "api_key":  API_KEY
#     }
#     print(f"▶️ Fetching {name} …")
#     r = requests.get(url, params=params, timeout=30)
#     r.raise_for_status()
#     data = r.json()  # list of {"t":<unix>, "v":<value>}
#     # ubah ke DataFrame
#     df = pd.DataFrame(data)
#     df["timestamp"] = pd.to_datetime(df["t"], unit="s")
#     df = df.rename(columns={"v": name}).drop(columns=["t"])
#     # simpan
#     df.to_csv(f"data/{name}.csv", index=False)
#     print(f"✅ Saved data/{name}.csv ({len(df)} rows)")

# print("🎉 Semua data on-chain per jam berhasil diambil ke folder `data/`")
